In [9]:
import re
import pandas as pd
from pathlib import Path

# ============================================================
# Dialogue Preprocessor — v2.1 (요약 모델 최적화 버전)
# ============================================================
class DialoguePreprocessorV2_1:
    def __init__(self, use_title_norm: bool = False):
        self.use_title_norm = use_title_norm

        self.title_map = {
            r'\bDr\.?\b': 'Dr.',
            r'\bMr\.?\b': 'Mr.',
            r'\bMrs\.?\b': 'Mrs.',
            r'\bMs\.?\b': 'Ms.',
            r'\bMiss\.?\b': 'Miss',
        }

        self.korean_title_map = {
            r'\b의사\s*선생님\b': '의사 선생님',
            r'\b교수님\b': '교수님',
            r'\b간호사\b': '간호사',
            r'\b상담사\b': '상담사',
            r'\b오빠\b': '오빠',
            r'\b형\b': '형',
            r'\b언니\b': '언니',
            r'\b누나\b': '누나',
            r'\b아저씨\b': '아저씨',
            r'\b아줌마\b': '아줌마',
            r'\b사장님\b': '사장님',
            r'\b손님\b': '손님',
            r'\b환자\b': '환자',
            r'\b기사님\b': '기사님',
        }

    # ====================================================
    # 1) newline / <br> / \\n 처리
    # ====================================================
    def normalize_newlines(self, text: str) -> str:
        text = re.sub(r'<br\s*/?>', '\n', text, flags=re.IGNORECASE)
        text = text.replace("\\n", "\n")
        text = re.sub(r'\n\s*\n+', '\n', text)
        return text

    # ====================================================
    # 2) speaker 태그 정규화 + 강제 줄바꿈
    # ====================================================
    def normalize_speaker(self, text: str) -> str:

        # (#person1# → #Person1#)
        text = re.sub(
            r'#\s*person\s*(\d+)\s*#\s*',
            lambda m: f'#Person{m.group(1)}#',
            text,
            flags=re.IGNORECASE
        )

        # colon 없는 경우 보정
        text = re.sub(r'(^|\n)(#Person\d+#)(?!:)', r'\1\2:', text)

        # *** 핵심 기능: speaker 등장 시 항상 줄바꿈 ***
        text = re.sub(r'\s*(#Person\d+#:)', r'\n\1', text)

        # 첫 줄 앞 \n 삭제
        text = text.lstrip("\n")

        return text

    # ====================================================
    # 3) 중간 호명 제거 안 함 (정보 보존)
    # ====================================================
    def remove_midline_speaker(self, text: str) -> str:
        return text

    # ====================================================
    # 4) 따옴표 정규화
    # ====================================================
    def normalize_quotes(self, text: str) -> str:
        quote_map = {
            "“": '"', "”": '"',
            "‘": "'", "’": "'",
            "「": '"', "」": '"',
            "『": '"', "』": '"',
            "‹": "'", "›": "'",
            "«": '"', "»": '"',
        }
        for k, v in quote_map.items():
            text = text.replace(k, v)
        return text

    # ====================================================
    # 5) title normalization (옵션)
    # ====================================================
    def normalize_titles(self, text: str) -> str:
        if not self.use_title_norm:
            return text
        for pat, repl in self.title_map.items():
            text = re.sub(pat, repl, flags=re.IGNORECASE)
        for pat, repl in self.korean_title_map.items():
            text = re.sub(pat, repl)
        return text

    # ====================================================
    # 6) 특수문자 제거
    # ====================================================
    def clean_hanja_noise(self, text: str) -> str:
        text = re.sub(r"[『』《》]", "", text)
        text = re.sub(r"[─━│┃╭╮╰╯]", "", text)
        return text

    # ====================================================
    # 7) 공백 정리
    # ====================================================
    def strip_spaces(self, text: str) -> str:
        lines = text.split("\n")
        lines = [ln.rstrip() for ln in lines]
        return "\n".join(lines)

    # ====================================================
    # 전체 파이프라인
    # ====================================================
    def run(self, text):
        if not isinstance(text, str):
            return ""

        text = self.normalize_newlines(text)
        text = self.normalize_speaker(text)
        text = self.remove_midline_speaker(text)
        text = self.normalize_quotes(text)
        text = self.normalize_titles(text)
        text = self.clean_hanja_noise(text)
        text = self.strip_spaces(text)

        return text



# ============================================================
# 실행
# ============================================================
if __name__ == "__main__":
    PROJECT_ROOT = Path.cwd().parents[1]
    RAW_DIR = PROJECT_ROOT / "data" / "raw"
    PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
    PROCESSED_DIR.mkdir(exist_ok=True)

    print("\nLoading raw CSV...")
    train_df = pd.read_csv(RAW_DIR / "train.csv")
    test_df = pd.read_csv(RAW_DIR / "test.csv")

    # ❗ FIX: 클래스명 수정
    pre = DialoguePreprocessorV23(use_title_norm=False)

    print("\nProcessing TRAIN...")
    train_df["dialogue_clean"] = train_df["dialogue"].apply(pre.run)

    print("Processing TEST...")
    test_df["dialogue_clean"] = test_df["dialogue"].apply(pre.run)

    train_out = PROCESSED_DIR / "v2_train_preprocessed.csv"
    test_out = PROCESSED_DIR / "v2_test_preprocessed.csv"

    train_df.to_csv(train_out, index=False)
    test_df.to_csv(test_out, index=False)

    print("\nSAVED:")
    print(train_out)
    print(test_out)
    print("\nDONE.")



Loading raw CSV...

Processing TRAIN...
Processing TEST...

SAVED:
/root/nlp/data/processed/v2_train_preprocessed.csv
/root/nlp/data/processed/v2_test_preprocessed.csv

DONE.


In [6]:
import difflib
from IPython.display import HTML, display

# ---------------------------------------------------
# HTML diff 함수 정의
# ---------------------------------------------------
def html_diff(a, b):
    differ = difflib.HtmlDiff(wrapcolumn=80)
    return HTML(differ.make_table(
        fromlines=a.split("\n"),
        tolines=b.split("\n"),
        fromdesc='RAW',
        todesc='CLEANED'
    ))

# ---------------------------------------------------
# 차이가 있는 행만 필터링
# ---------------------------------------------------
diff_df = train_df[train_df["dialogue"] != train_df["dialogue_clean"]]

# ---------------------------------------------------
# 앞 5개 샘플 HTML diff로 표시
# ---------------------------------------------------
for idx in diff_df.head(10).index:
    print(f"\n==== SAMPLE {idx} ====\n")
    display(html_diff(
        diff_df.loc[idx, "dialogue"],
        diff_df.loc[idx, "dialogue_clean"]
    ))



==== SAMPLE 0 ====




==== SAMPLE 11 ====




==== SAMPLE 12 ====




==== SAMPLE 16 ====




==== SAMPLE 18 ====




==== SAMPLE 21 ====




==== SAMPLE 23 ====




==== SAMPLE 27 ====




==== SAMPLE 29 ====




==== SAMPLE 35 ====



In [12]:
# ============================================================
# 줄바꿈 시각화 + Raw/Clean 비교 유틸
# ============================================================

def show_with_linebreaks(text, max_len=800):
    """줄바꿈을 ⏎ 로 표시해서 눈에 보이게 변환."""
    if not isinstance(text, str):
        print(text)
        return
    
    view = text.replace("\n", "⏎\n")
    if len(view) > max_len:
        view = view[:max_len] + "\n... (truncated) ..."
    print(view)


def compare_linebreaks(df, idx, raw_col="dialogue", clean_col="dialogue_clean"):
    print(f"\n\n========== SAMPLE {idx} ==========\n")

    print("===== RAW =====")
    show_with_linebreaks(df.loc[idx, raw_col])

    print("\n===== CLEANED =====")
    show_with_linebreaks(df.loc[idx, clean_col])


def compare_multiple(df, indices, raw_col="dialogue", clean_col="dialogue_clean"):
    for idx in indices:
        compare_linebreaks(df, idx, raw_col, clean_col)


# ============================================================
# 실행 예시 — 전처리 코드 내부에서 바로 작동하도록 수정
# ============================================================
if __name__ == "__main__":

    # 이미 위에서 PROJECT_ROOT, PROCESSED_DIR 만들어 둠
    processed_train_path = PROCESSED_DIR / "v2_train_preprocessed.csv"

    print("Loading processed file:", processed_train_path)

    df = pd.read_csv(processed_train_path)

    # 확인하고 싶은 index들
    samples = [0, 3, 11, 12, 18, 21, 27]

    compare_multiple(df, samples)


Loading processed file: /root/nlp/data/processed/v2_train_preprocessed.csv


========== SAMPLE 0 ==========

===== RAW =====
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? ⏎
#Person2#: 건강검진을 받으려고 왔어요. ⏎
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. ⏎
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. ⏎
#Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요. ⏎
#Person2#: 알겠습니다. ⏎
#Person1#: 여기 좀 볼까요. 눈과 귀는 괜찮으시네요. 깊게 숨 한 번 쉬어보세요. Mr. Smith, 담배 피우세요? ⏎
#Person2#: 네. ⏎
#Person1#: 담배가 폐암하고 심장병의 주된 원인인 거 아시죠? 끊으셔야 해요. ⏎
#Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요. ⏎
#Person1#: 음, 도움 될만한 수업과 약물들이 있습니다. 가시기 전에 더 정보를 드릴게요. ⏎
#Person2#: 네, 고맙습니다, 의사 선생님.

===== CLEANED =====
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요?⏎
#Person2#: 건강검진을 받으려고 왔어요.⏎
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요.⏎
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요.⏎
#Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요.⏎
#Person2#: 알겠습니다.⏎
#Person1#: 여기 좀 볼까